### study

- reflexion is a paradigm that uses **verbal reinforcement** that gets encoded in an agent's memory.
- reflexion converts feedback from the environment into linguistic feedback, (like self-reflection) which is inserted into context for an LLM agent in the next episode. This helps the agent rapidly and effectively learn from prior mistakes leading to performance improvements on many advanced tasks.
- three distinct models:
    - **actor**: generates text+actions. use CoT, ReAct for this.
    - **evaluator**: score outputs produced by actor. different rewarding functions are used depending on the task (llm and rule-based heuristics are used for decision-making tasks).
    - **self-reflection**: generates verbal reinforcement cues to assist the Actor in self-improvement. uses reward signal + current trajectory + persistent memory.
- steps:
    - define a task
    - generate a trajectory
    - evaluate
    - perform reflection
    - generate the next trajectory

![Reflexion](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Freflexion-examples.7558c279.png&w=3840&q=75)


- effective:
    - sequential decision making
    - reasoning
    - programming

- limitations:
    - reliance on self-evaluation capabilities
    - long-term memory constraints
    - code generation limitations

Sources: https://www.promptingguide.ai/techniques/reflexion

- post outlines how to build 3 reflection techniques using LangGraph, including implementations of Reflexion and Language Agent Tree Search
    - simple reflection 
    - reflexion
    - language agents tree search

- thinking types: **System 1**, **System 2**
    - System 1: reactive, instictive
    - System 2: methodical, reflective
    - what we want : System 2 like behavior
    - how: whne applied correctly, reflection can help LLM systems break out of purely System 1 **thinking** patterns and closer to something exhibiting System 2-like behavior.
     - https://neurofied.com/wp-content/uploads/2022/05/System-12-White-1.png


Sources: 
- https://blog.langchain.dev/reflection-agents/
- https://github.com/langchain-ai/langgraph/blob/main/examples/reflexion/reflexion.ipynb

### implement

Thoughts:
1. trying to learn from https://github.com/noahshinn/reflexion
2. actor, evaluator, self-reflection
3. write prompts for roles: actor, evaluator, self-reflection
4. write output_parsers for above
5. execute
    - run agent (actor)
    - evaluate final_answer from actor using evaluator
    - self-reflect with trajectory from actor + eval from evaluator to give relfection
    - run agent till the evaluator gives correct answer or n (turns to run this while loop) exhausted

reference: https://github.com/aasthavar/reflexion/blob/main/hotpotqa_runs/prompts.py
1. cot, cot-reflect
2. react, react-reflect

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import boto3
from langchain_community.chat_models import BedrockChat
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import get_buffer_string
from utils.reflextion_agent import *

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(name)s | L:%(lineno)d | %(message)s",
    force=True
)
logger = logging.getLogger(__name__)
logger.info("Invoking main_agent")

In [ ]:
#### test agent
# main_agent = get_main_agent()
# agent_result = main_agent_executor.invoke({
#     "question": "Which character does this protagonist, who secretly loves and marries a member of the rival house, of William Shakespeare's tragedy that has a fictional character Benvolio slay?",
#     "reflections": ""
# })

#### test eval
# evaluator_agent = get_evaluator_agent()
# eval_result = evaluator_agent.invoke({
#     # "answer": "Benvolio", # 0
#     # "answer": "Tybalt", # 1
#     "answer": "Juliet", # 0 # this is a little buggy
#     "question": "Which character does this protagonist, who secretly loves and marries a member of the rival house, of William Shakespeare's tragedy that has a fictional character Benvolio slay?"
# })

#### test self-reflect
# reflect_chain = get_reflexion_chain()
# main_agent_scratchpad = get_main_agent_scratchpad(
#     intermediate_steps=agent_result["intermediate_steps"], 
#     final_answer=agent_result["output"]
# )
# print(main_agent_scratchpad)
# reflect_result = self_reflection_chain.invoke({
#     "question": "Which character does this protagonist, who secretly loves and marries a member of the rival house, of William Shakespeare's tragedy that has a fictional character Benvolio slay?",
#     "scratchpad": main_agent_scratchpad,
# })

In [15]:
# question = "Which character does this protagonist, who secretly loves and marries a member of the rival house, of William Shakespeare's tragedy that has a fictional character Benvolio slay?",
# question = """The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes?"""
# question = "The head of the Foreign Relations Department of the Rastriya Janashakti Party holds a degree that can be abbreviated MS, M.S., or ScM, in what field?"
# question = """What is represented on the cover art of the studio album that includes the song "Speed of Sound"?"""


reflections = []
evaluate_final_answer = 0
main_agent_intermediate_steps = ""
num_of_steps = 1


while (evaluate_final_answer!=1) and (num_of_steps<=3):
    main_agent = get_main_agent()
    logger.info(f"step: {num_of_steps}. Invoking main_agent")
    main_agent_result = main_agent.invoke({
        "reflections": format_reflections(reflections),
        "question": question,
    })
    
    evaluator_agent = get_evaluator_agent()
    main_agent_final_answer = main_agent_result["output"]
    logger.info(f"step: {num_of_steps}. Invoking evaluator_agent")
    evaluator_result = evaluator_agent.invoke({
        "answer": main_agent_final_answer,
        "question": question,
    })
    
    logger.info(f"evaluator_result_output: {evaluator_result['output']}")
    logger.info(f"main_agent_final_answer: {main_agent_final_answer}")
    
    evaluate_final_answer = 1 if "1" in evaluator_result["output"] else 0
    if evaluate_final_answer == 1:
        break

    reflect_chain = get_reflexion_chain()
    main_agent_scratchpad = get_main_agent_scratchpad(
        intermediate_steps=main_agent_result["intermediate_steps"], 
        final_answer=main_agent_result["output"]
    )
    logger.info(f"step: {num_of_steps}. Invoking reflect_chain")
    reflect_chain_results = reflect_chain.invoke({
        "question": question,
        "scratchpad": main_agent_scratchpad,
    })
    
    reflections.append(reflect_chain_results)
    logger.info(f"reflections: {reflections}")
    
    num_of_steps += 1

logger.info(f"after while loop exit, main_agent_final_answer: {main_agent_final_answer}")

__main__ | L:15 | step: 1. Invoking main_agent




> Entering new AgentExecutor chain...
Question: What is represented on the cover art of the studio album that includes the song "Speed of Sound"?
Thought: The question is asking about the cover art of a specific album that contains the song "Speed of Sound". To answer this, I will need to identify the album and look up information about its cover art.
Action: search-wikipedia
Action Input: "Speed of Sound" songPage: Speed of Sound (song)
Summary: "Speed of Sound" is a song by British rock band Coldplay. It was written by all members of the band for their third studio album, X&Y (2005). Constructed around a piano and guitar riff, the song builds into a huge, synthesiser-heavy chorus. It was released by Parlophone Records as the lead single from the album. "Speed of Sound" made its radio premiere on BBC Radio 1 with Lamacq on 19 April, then was serviced to US radio on 18 April 2005. The song was released physically in Japan on 11 May with two B-sides: "Things I Don't Understand" and "P

__main__ | L:23 | step: 1. Invoking evaluator_agent


Thought: The Wikipedia page for the X&Y album provides the key information needed to answer the question. It states that the album cover art "combines colours and blocks to represent the title in Baudot code."

Final Answer: The cover art of Coldplay's 2005 album X&Y, which includes the song "Speed of Sound", represents the album title in Baudot code using colors and blocks.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: I need more information to verify the reference answer.
Action: wikipedia
Action Input: Coldplay X&Y album coverPage: X&Y
Summary: X&Y is the third studio album by British rock band Coldplay. It was released on 6 June 2005 by Parlophone in the United Kingdom, and a day later by Capitol Records in the United States. The album was produced by Coldplay and producer Danton Supple. It is noted for its troubled and urgent development, as well as manager and creative director Phil Harvey's brief departure from the band. Producer Ken Nelson was originally 

__main__ | L:29 | evaluator_result_output: 1
__main__ | L:30 | main_agent_final_answer: The cover art of Coldplay's 2005 album X&Y, which includes the song "Speed of Sound", represents the album title in Baudot code using colors and blocks.
__main__ | L:52 | after while loop exit, main_agent_final_answer: The cover art of Coldplay's 2005 album X&Y, which includes the song "Speed of Sound", represents the album title in Baudot code using colors and blocks.


Final Answer: 1

> Finished chain.
